# **Data Augmentation**

In [1]:
# %pip install tensorflow
%pip install opencv-python --upgrade 


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# transformation_array -> User can choose what transformations to apply. Simply comment/uncomment (#) the row corresponding to the transformation name.
# enable_show -> User can choose to display (True) or none (False) the transformations applied to original images. Enabling this option, the execution speed will be reduced.

transformation_array = [
                        "horizontalFlip",
                        "verticalFlip",
                        "rotation",
                        "widthShift",
                        "heightShift"
                        # ,
                        # "shearRange",
                        # "zoom",
                        # "blur",
                        # "brightness",
                        # "contrast",
                        # "saturation",
                        # "hue",
                        # "gamma"
                        ];
enable_show = False;

In [2]:
# The new dataset 'augmented_waste_dataset' will be created.
# This dataset contains the augmented images create by the ImageGenerator class and the orginal images,
# in order to obtain an expanded version of the orginal dataset ready-to-use

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from numpy import expand_dims
import cv2
import matplotlib.pyplot as plt
from pathlib import Path



def blur(img):
    return (cv2.blur(img,(30,30)))

def horizontal_flip(img):
    return (tf.image.flip_left_right(img))

def vertical_flip(img):
    return (tf.image.flip_up_down(img))

def contrast(img):
    return (tf.image.adjust_contrast(img, 0.5))

def saturation(img):
    return (tf.image.adjust_saturation(img, 3))

def hue(img):
    return (tf.image.adjust_hue(img, 0.1))

def gamma(img):
    return (tf.image.adjust_gamma(img, 2))

# modified dataset source: starting with 100 images
classes = ['metal', 'other']
dataset_name = f'{classes[0]}_and_{classes[1]}'
new_dataset = f'augmented_{classes[0]}_and_{classes[1]}' 
for class_tag in classes:
  input_path = '/app/data/Datasets/ ' + dataset_name + '/' + class_tag + '/'
  output_path = '/app/data/Datasets/' + new_dataset + '/' + class_tag + '/'
  print(input_path)
  print(output_path)
#   # TMP
#   !rm -rf $output_path
#   # END TMP
  # Skip augmentation for 'other' class
  

  print(f"Processing {class_tag} class.\n")

  try:
    if not os.path.exists(output_path):
      os.makedirs(output_path)
  except OSError:
      print ("Creation of the directory %s failed\n\n" % output_path)
  else:
      print ("Successfully created the directory %s\n\n" % output_path)

  for filename in os.listdir(input_path):
    if filename.endswith(".jpg"):
      # Copy the original image in the new dataset
      original_file_path = input_path + filename
      original_newname_file_path = output_path + Path(filename).stem + "_original.jpg"
      %cp $original_file_path $original_newname_file_path
      # Initialising the ImageDataGenerator class.
      # We will pass in the augmentation parameters in the constructor.
      if class_tag == 'other':
       print(f"Skipping augmentation for {class_tag} class.\n")
       continue
      for transformation in transformation_array:
        if transformation == "horizontalFlip":
              #datagen = ImageDataGenerator(horizontal_flip = True)                 # for random flip
              datagen = ImageDataGenerator(preprocessing_function=horizontal_flip)  # all imgs flipped
        elif transformation == "verticalFlip":
              #datagen = ImageDataGenerator(vertical_flip = True)                   # for random flip
              datagen = ImageDataGenerator(preprocessing_function=vertical_flip)    # all imgs flipped
        elif transformation == "rotation":
              datagen = ImageDataGenerator(rotation_range = 40, fill_mode='nearest')
        elif transformation == "widthShift":
              datagen = ImageDataGenerator(width_shift_range = 0.2, fill_mode='nearest')
        elif transformation == "heightShift":
              datagen = ImageDataGenerator(height_shift_range = 0.2, fill_mode='nearest')
        elif transformation == "shearRange":
              datagen = ImageDataGenerator(shear_range = 0.2)
        elif transformation == "zoom":
              datagen = ImageDataGenerator(zoom_range = [0.5, 1.0])
        elif transformation == "blur":
              datagen = ImageDataGenerator(preprocessing_function=blur)
        elif transformation == "brightness":
              #Values less than 1.0 darken the image, e.g. [0.5, 1.0],
              #whereas values larger than 1.0 brighten the image, e.g. [1.0, 1.5],
              #where 1.0 has no effect on brightness.
              datagen = ImageDataGenerator(brightness_range = [1.1, 1.5])
        elif transformation == "contrast":
              datagen = ImageDataGenerator(preprocessing_function=contrast)
        elif transformation == "saturation":
              datagen = ImageDataGenerator(preprocessing_function=saturation)
        elif transformation == "hue":
              datagen = ImageDataGenerator(preprocessing_function=hue)
        elif transformation == "gamma":
              datagen = ImageDataGenerator(preprocessing_function=gamma)

        # Loading a sample image and convert to RGB if not already RGB
        img = load_img(input_path + filename)

        # Converting the input sample image to an array
        data = img_to_array(img)
        # Reshaping the input image expand dimension to one sample
        samples = expand_dims(data, 0)
        # Plot original image
      #   print("Original image:")
      #   print(filename)
        if enable_show:
          plt.imshow(img)
          plt.show()
          print("\n\n")

        # Generating and saving n_augmented_images augmented samples
        print("Apply " + transformation + ".")
        # prepare iterator
        it = datagen.flow(samples, batch_size = 1,
                    save_to_dir = output_path,
                    save_prefix = Path(filename).stem + "_" + transformation,
                    save_format ='jpg')
        batch = next(it)    #corrected from batch = it.next()
        # Plot trasnformed image
        image = batch[0].astype('uint8')
        if enable_show:
          print("Transformed image:")
          plt.imshow(image)
          plt.show()
        print("\n\n")

print("Done!\n\n")

2024-09-17 15:36:00.191550: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 15:36:00.200500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 15:36:00.211526: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 15:36:00.214763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 15:36:00.222602: I tensorflow/core/platform/cpu_feature_guar

/app/data/Datasets/ metal_and_other/metal/
/app/data/Datasets/ augmented_metal_and_other/metal/
Processing metal class.

Successfully created the directory /app/data/Datasets/ augmented_metal_and_other/metal/




FileNotFoundError: [Errno 2] No such file or directory: '/app/data/Datasets/ metal_and_other/metal/'